In [1]:
import nibabel as nib

md_path = 'Sub2_MD.nii'
atlas_path = 'Sub2_AtlasBNA.nii'

md_img = nib.load(md_path)
atlas_img = nib.load(atlas_path)

md_data = md_img.get_fdata()
atlas_data = atlas_img.get_fdata()


In [2]:
import numpy as np


def mean_of_non_zero_neighbors(md_data, atlas_data, x, y, z):

    neighbor_values = []
    

    for i in range(max(0, x-1), min(md_data.shape[0], x+2)):
        for j in range(max(0, y-1), min(md_data.shape[1], y+2)):
            for k in range(max(0, z-1), min(md_data.shape[2], z+2)):

                if (i, j, k) != (x, y, z) and atlas_data[i, j, k] != 0:
                    neighbor_values.append(md_data[i, j, k])
    

    if neighbor_values:
        return np.mean(neighbor_values)
    else:
        return md_data[x, y, z] 


md_data_adjusted = np.copy(md_data)


for x in range(md_data.shape[0]):
    for y in range(md_data.shape[1]):
        for z in range(md_data.shape[2]):

            if atlas_data[x, y, z] != 0:
                md_data_adjusted[x, y, z] = mean_of_non_zero_neighbors(md_data, atlas_data, x, y, z)


In [3]:
from scipy import stats

original_md_flat = md_data.flatten()
transformed_md_flat = md_data_adjusted.flatten()


mean_original = np.mean(original_md_flat)
mean_transformed = np.mean(transformed_md_flat)


t_stat, p_value = stats.ttest_ind(original_md_flat, transformed_md_flat, equal_var=False)

print(f"Mean Original MD Values: {mean_original}")
print(f"Mean Transformed MD Values: {mean_transformed}")
print(f"T-statistic: {t_stat}, P-value: {p_value}")


Mean Original MD Values: 0.00017184854368577245
Mean Transformed MD Values: 0.00017189893330785152
T-statistic: -0.1749414444630061, P-value: 0.8611256513425116
